In [ ]:
include("./BBTNet/datasets/WIDERFACE.jl")

root_dir = "../Datasets/WIDERFACE/WIDER_"
train_data = WIDER_Data(root_dir * "train/")
print("WIDER Data is read. Total images: ", train_data.num_files, " & Total Faces: ", train_data.num_faces, '\n')

In [ ]:
include("./BBTNet/datasets/WIDERFACE.jl")

(imgs, boxes), state = iterate(train_data)
print("Got the first batch: ", size(imgs))

In [ ]:
include("./BBTNet/utils/ImageReader.jl")

train_reader = Image_Reader(true)
test_reader = Image_Reader(false)

filename = "0--Parade/0_Parade_marchingband_1_799.jpg"
img, bboxes = read_img(train_reader, root_dir * "train/images/" * filename, train_data.bboxes[filename], 640)
print("Image file is read: ", filename, '\n')

In [ ]:
include("./BBTNet/utils/draw.jl")

draw_boxes_and_landmarks(img, bboxes)

## Running The Model

In [ ]:
include("./BBTNet/models/retinaface.jl")

# atype = KnetArray{Float32}
atype = Array{Float32}

x = convert(atype, rand(640, 640, 3, 2))
model = RetinaFace(dtype=atype)

c, b, l = model(x, train=false)
print(size(c), " & ", size(b), " & ", size(l), " \n")

In [ ]:
include("BBTNet/models/retinaface.jl")
include("BBTNet/datasets/WIDERFACE.jl")
include("./BBTNet/utils/draw.jl")
include("./BBTNet/utils/box_processes.jl")

atype = KnetArray{Float32}
# atype = Array{Float32}
images_folder_dir = "../Datasets/WIDERFACE/WIDER_"

val_data = WIDER_Data(images_folder_dir * "val/", train=false, batch_size=2)
(imgs, boxes), state = iterate(val_data)
print("Got the first batch: ", size(imgs), "\n")

model = RetinaFace(dtype=atype)
imgs_permuted = convert(atype, permutedims(imgs, (3,2,1,4)))
c, b, l = model(imgs_permuted, train=false)

combined = permutedims(cat(b[1], l[1], dims=2), (2, 1))

draw_boxes_and_landmarks(imgs[:,:,:,1], combined)

In [ ]:
using JLD2
using FileIO

@save "retinaface_rand.jld2" model
model_dict = load("retinaface_rand.jld2")
model2 = model_dict["model"]
print("Model is loaded from a file!\n")

In [1]:
import DataStructures

In [ ]:
import Knet

In [7]:
temp = rand(10)
i = sortperm(temp)
typeof(temp)

Array{Float64,1}

In [6]:
heap = BinaryHeap(Base.By(last), temp)
maxs = popmax!(heap, 3)
typeof(maxs)

LoadError: UndefVarError: BinaryHeap not defined

In [ ]:
max_prior_vals, max_prior_idx = findmax(temp; dims=2)
print(max_prior_vals)

In [ ]:
getindex.(max_prior_idx[findall(max_prior_vals .>= 0.85)], [1 2])[:, 2]

In [ ]:
pos_indices = findall(max_prior_vals .>= 0.7)
size(pos_indices)

In [ ]:
getindex.(pos_indices, [1 2])[:,1]

In [ ]:
vals, idx = sort(max_prior_vals, dims=2)

In [ ]:
vals

In [ ]:
num_anchors     = 3
anchor_info     = [ 
    Dict([("stride", 4), ("anchors", [16, 20.16, 25.40]), ("end_idx", 76800)]),
    Dict([("stride", 8), ("anchors", [32, 40.32, 50.80]), ("end_idx", 96000)]),
    Dict([("stride", 16), ("anchors", [64, 80.63, 101.59]), ("end_idx", 100800)]),
    Dict([("stride", 32), ("anchors", [128, 161.26, 203.19]), ("end_idx", 102000)]),
    Dict([("stride", 64), ("anchors", [256, 322.54, 406.37]), ("end_idx", 102300)])
]

crop_ratios     = [0.3, 0.45, 0.6, 0.8, 1.0]
img_size        = 640

function _get_priorboxes(;dtype=Array{Float64})
    feature_maps = [Int(ceil(img_size / scale["stride"])) for scale in anchor_info]
    num_proposals = num_anchors * sum([i*i for i in feature_maps])
    anchors = convert(dtype, zeros(num_proposals, 4))

    counter = 1
    for (idx, f) in enumerate(feature_maps)
        scaler = anchor_info[idx]["stride"] / img_size
        for s in anchor_info[idx]["anchors"]
            bbox_len = s / img_size
            for h in 1:f
                for w in 1:f
                    cx = (w - 0.5) * scaler
                    cy = (h - 0.5) * scaler
                    anchors[counter,:] = convert(dtype, [cx cy bbox_len bbox_len])
                    counter += 1
                end
            end
        end
    end
    return anchors
end

anchors = _get_priorboxes()

In [ ]:
temp2  =[-2 5; -1 1]
abs.(temp2)

In [ ]:
sum(temp2)